In [ ]:
!pip install wandb

In [18]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import cv2
import wandb
from wandb.keras import WandbCallback

In [3]:
wandb.init(project='mnist')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
config = wandb.config
config.learning_rate = 0.001

## dataset

In [11]:
mnist = tf.keras.datasets.mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size= 0.1)

# normalizaition
X_train = X_train / 255.0
X_test = X_test / 255.0

## CNN + MLP

In [12]:
model = tf.keras.models.Sequential([
    #استخراج ویژگی
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (5, 5), activation='relu'),

    layers.Flatten(),

    #همون شبکه ساده قدیمی
    layers.Dense(48, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [14]:
model.compile(optimizer= tf.keras.optimizers.Adam(config.learning_rate),
              loss= tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy']
)

In [15]:
model.fit(X_train, Y_train, epochs=5, validation_data=(X_val, Y_val), callbacks=[WandbCallback()])

Epoch 1/5
1685/1688 [============================>.] - ETA: 0s - loss: 0.1712 - accuracy: 0.9468

wandb: Adding directory to artifact (/content/wandb/run-20230714_124415-n4t1jtd9/files/model-best)... Done. 0.0s


1688/1688 [==============================] - 39s 22ms/step - loss: 0.1710 - accuracy: 0.9469 - val_loss: 12.3161 - val_accuracy: 0.9798
Epoch 2/5
1687/1688 [============================>.] - ETA: 0s - loss: 0.0540 - accuracy: 0.9830

wandb: Adding directory to artifact (/content/wandb/run-20230714_124415-n4t1jtd9/files/model-best)... Done. 0.0s


1688/1688 [==============================] - 38s 23ms/step - loss: 0.0540 - accuracy: 0.9830 - val_loss: 11.4495 - val_accuracy: 0.9788
Epoch 3/5
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0381 - accuracy: 0.9880 - val_loss: 12.3448 - val_accuracy: 0.9805
Epoch 4/5
1688/1688 [==============================] - 38s 22ms/step - loss: 0.0284 - accuracy: 0.9906 - val_loss: 25.0806 - val_accuracy: 0.9675
Epoch 5/5
1688/1688 [==============================] - 35s 21ms/step - loss: 0.0222 - accuracy: 0.9928 - val_loss: 22.6025 - val_accuracy: 0.9728


In [16]:
model.save('CNN_mnist.h5')

In [17]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.0508 - accuracy: 0.9842


[0.05076028034090996, 0.9842000007629395]

In [19]:
img = cv2.imread('7.jpg')
img = cv2.resize(img, (28,28))
img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

img = img / 255.0
img = img.reshape(1, 28, 28)

result = model.predict(img)
np.argmax(result)

1/1 [==============================] - 0s 133ms/step


7

## MLP

In [21]:
model = tf.keras.models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [22]:
model.compile(optimizer= 'Adam',
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [24]:
model.fit(X_train, Y_train, epochs=5, callbacks=[WandbCallback()])

Epoch 1/5
1688/1688 [==============================] - 4s 3ms/step - loss: 0.1624 - accuracy: 0.9518
Epoch 2/5
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1021 - accuracy: 0.9695
Epoch 3/5
1688/1688 [==============================] - 4s 3ms/step - loss: 0.0736 - accuracy: 0.9777
Epoch 4/5
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0564 - accuracy: 0.9825
Epoch 5/5
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0428 - accuracy: 0.9870


In [25]:
model.save('MLP_mnist.h5')

In [26]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0700 - accuracy: 0.9788


[0.06997055560350418, 0.9787999987602234]